This ledger contains all updates to the project. It includes important code snippets, information, explantion, references and other relavant information for all collaborators. 

# Date: 11/17/2019

Inital setup
* Created ledger
* Created git repository
* Shifted ledger to git 

Added nntools.py to folder src
* Use to inherit Experiment and other class funtionalities

Created a new folder: local_tools
* This folder contains all the custom libraries and classes that we write
* Shifted nntools.py to this folder

Created a new folder: local_notebooks
* This folder used to create different local python notebooks for each colaborators
* Any changes with this 

### Command to download a file (e.g. a model) from Google Drive

wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=FILEID' -O FILENAME

E.g: wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1b-_sjq1Pe_dVxt5SeFmoadMfiPTPZqpz' -O ctdet_coco_resdcn18.pth

### Make external
~/MoDL_CenterNet/src/lib/external$ make

### DCNv2 for Pytorch 1.0
Repo: https://github.com/CharlesShang/DCNv2 (clone in MoDL_CenterNet/src/lib/models/networks/DCNv2/)

python setup.py build  # build

PYTHONPATH="${PYTHONPATH}:path_until_here/MoDL_CenterNet/src/lib/models/networks/DCNv2"

export PYTHONPATH

python setup.py develop --instal-dir path_until_here/MoDL_CenterNet/src/lib/models/networks/DCNv2 #install

### For Test script error on unable to find model
Copy the model to MoDL_CenterNet/exp/ctdet/model_name and rename to 'model_last.pth